In [1]:
import pandas as pd
import numpy as np
def write_submission_file(_model, _featureList): # see submission.ipynb for details
    import pandas as pd
    sample = pd.read_csv('data/sample.csv', index_col=0)
    Xsample = sample[_featureList].values
    sample['Pred'] = _model.predict_proba(Xsample)[:,0]
    submission = sample[['Id', 'Pred']]
    submission.to_csv("data/submission.csv", encoding='ascii')
    
tourney_df = pd.read_csv("data/data.csv", index_col=0)
tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lstl,Lblk,Lpf,Wteam_name,Lteam_name,Winner,team0,team1,wins0,wins1
0,2003,134,1421,92,1411,84,N,1,32,69,...,5,0,22,UNC Asheville,TX Southern,0,1421,1411,13,18
1,2003,136,1112,80,1436,51,N,0,31,66,...,10,3,15,Arizona,Vermont,1,1436,1112,19,25
2,2003,136,1113,84,1272,71,N,0,31,59,...,2,5,18,Arizona St,Memphis,1,1272,1113,23,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,6,6,21,C Michigan,Creighton,0,1141,1166,23,29
4,2003,136,1143,76,1301,74,N,1,27,64,...,5,8,19,California,NC State,1,1301,1143,18,21


# Choosing Features to Use for Model

Edit featureList below to use any of the features in the data frame above (see above for names of columns).  

In [3]:
featureList = ['wins0', 'wins1']
y = tourney_df['Winner'].values # results
X = tourney_df[featureList].values # features

# Train our model!

In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [5]:
from sklearn import svm
model = svm.SVC(probability=True)
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [6]:
y_pred = model.predict_proba(X_test)[:,0] # probability that team0 wins (what Kaggle calls team 1, and wants for submission)
for i in range(5):
    print("Features = {0}, Pred. prob. team0 wins: {1}".format(X_test[i], y_pred[i]))

Features = [30 26], Pred. prob. team0 wins: 0.48888984248377665
Features = [25 33], Pred. prob. team0 wins: 0.4218629377446364
Features = [32 27], Pred. prob. team0 wins: 0.5
Features = [24 22], Pred. prob. team0 wins: 0.3755771294506705
Features = [26 29], Pred. prob. team0 wins: 0.49241673415501036


# Could you double-check I'm calculating this correctly?

Just realized Kaggle uses the convention for scoring that the label is 1 if the first team wins, and 0 if the second.  We use 0 if team0 wins, 1 if team1 wins.  I think what's below works...passing y_pred, i.e. the probability that the first team wins (i.e. what we call team0, what they call team 1), and 1-y_test, so that 1-y = 0 when team1 wins (their team2) and 1-y = 1 when team0 wins (their team1).  Please compare http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html and https://www.kaggle.com/c/march-machine-learning-mania-2016/details/evaluation to make sure we're doing the same thing.  It's late and I'm not thinking straight.

In [7]:
from sklearn import metrics
print("Log loss is {0}".format(metrics.log_loss(1-y_test, y_pred)))

Log loss is 0.6489059909396542


# Making a submission on Kaggle

The following call generates a file `data/submission.csv` that uses the model you pass it to generate probabilities for all the matches Kaggle wants us to predict.  It will overwrite any previous file.  See `submission.ipynb` for a walk-through of the function.  Have to pass it the model you trained, and the featureList you used.

In [8]:
write_submission_file(model, featureList)